In [1]:
import os, sys, re, pickle
import numpy as np
import pandas as pd

In [2]:
sys.path.insert(0, '/home/jgutman/mvesc/ETL')
sys.path.insert(0, '/home/jgutman/mvesc/Models_Results')

In [3]:
from mvesc_utility_functions import *

In [7]:
model_list = ["'RF'", "'logit'"]
batch_date = '08_17_2016'
table_name = '_'.join(['best_models', batch_date])
outcome = 'definite_plus_ogt'

with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        temp_explore_results = """
        drop table if exists {table};
        create temporary table {table} as
            select model_name, filename, prediction_grade, label, 
            val_set, val_recall_5_15, test_set, test_recall_5_15 
            from model.reports
            where filename in 
                (select distinct on (prediction_grade, model_name)
                filename from model.reports
                where filename like '{batch_date}%'
                and cv_criterion like 'custom_recall%'
                and model_name in ({model_list})
                and label like '{outcome}'
                order by prediction_grade, model_name, cv_score desc)
            and cv_criterion like 'custom_recall%'
            order by model_name, prediction_grade;
        """.format(table = table_name, batch_date=batch_date,
                      model_list = ', '.join(model_list), outcome = outcome)
        cursor.execute(temp_explore_results)
        
        get_val_test_years = """
        select prediction_grade, val_set, test_set
            from {table} where model_name = {first_model}
        """.format(table = table_name, first_model = model_list[0])
        cursor.execute(get_val_test_years)
        val_test_years = cursor.fetchall()
        
        store_outcome_counts = pd.DataFrame(index = [year[0] for year in val_test_years], 
                columns = ['val_set', 'val_neg', 'val_pos', 'val_pct_pos',
                           'test_set', 'test_neg', 'test_pos', 'test_pct_pos'])
        for grade, val, test in val_test_years:
            get_counts = """ select count({outcome})
                from model.outcome where cohort_{grade}th in ({val}, {test})
                and {outcome} is not null
                group by (cohort_{grade}th, {outcome})
                order by (cohort_{grade}th, {outcome});
            """.format(outcome = outcome, grade = grade, val = val, test = test)
            cursor.execute(get_counts)
            results = [i[0] for i in cursor.fetchall()]
            store_outcome_counts.loc[grade] = [val, results[0], results[1], 
                results[1]/(results[0]+results[1]), test, results[2], results[3],
                results[3]/(results[2]+results[3])]

In [8]:
store_outcome_counts

,val_set,val_neg,val_pos,val_pct_pos,test_set,test_neg,test_pos,test_pct_pos
6,2008,875,107,0.108961,2009,930,88,0.086444
7,2009,891,110,0.10989,2010,1036,117,0.101474
8,2010,1113,141,0.11244,2011,1801,226,0.111495
9,2011,1881,276,0.127955,2012,2037,281,0.121225
10,2012,1838,252,0.120574,2013,2034,252,0.110236
